In [ ]:
import dask.dataframe as dd

# Read the CSV file into a Dask DataFrame
df = dd.read_csv('data.csv')

# Filter the DataFrame to only include rows where DetectorID = 656.1.503
filtered_df = df[df['DetectorID'] == 656.1503]

# Print the filtered DataFrame
print(filtered_df)

In [ ]:
import csv
from pyspark.sql import SparkSession

# Read the detector IDs from the CSV file
detector_ids = []
with open('/home/tzahid/Desktop/Xie/data/detector_ids.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        detector_ids.append(row[0])

# Create a SparkSession
spark = SparkSession.builder.appName("filter_detectorid").getOrCreate()

# Read the FASTFreewaySensorData2018.csv file into a Spark DataFrame
df = spark.read.csv("/home/tzahid/Desktop/Xie/data/FASTFreewaySensorData2018.csv", header=True)

# Process each detector ID
for detector_id in detector_ids:
    # Convert the detector ID to the format with '.' instead of '_'
    normalized_detector_id = detector_id.replace('_', '.')

    # Filter the DataFrame to include rows where "DetectorID" is equal to the normalized detector ID
    filtered_df = df.filter(df["DetectorID"] == normalized_detector_id)

    # Reduce the number of partitions to one using coalesce()
    coalesced_df = filtered_df.coalesce(1)

    # Construct the output file path using the detector ID
    output_filepath = f"/home/tzahid/Desktop/Xie/data/{detector_id}/{detector_id}.csv"

    # Save the coalesced DataFrame as a CSV file with headers
    coalesced_df.write.option("header", True).mode("overwrite").csv(output_filepath)

# Stop the SparkSession
spark.stop()


In [ ]:
import csv

# Creating a memory efficient 

def process_detector_id(detector_id):
    # Normalize the detector ID
    normalized_detector_id = detector_id.replace('_', '.')

    # Create a SparkSession for this detector ID
    spark = SparkSession.builder.appName(f"filter_detectorid_{detector_id}").getOrCreate()

    # Read the FASTFreewaySensorData2018.csv file into a Spark DataFrame
    df = spark.read.csv("/home/tzahid/Desktop/Xie/data/FASTFreewaySensorData2018.csv", header=True)

    # Filter the DataFrame to include rows where "DetectorID" is equal to the normalized detector ID
    filtered_df = df.filter(df["DetectorID"] == normalized_detector_id)

    # Reduce the number of partitions to one using coalesce()
    coalesced_df = filtered_df.coalesce(1)

    # Construct the output file path using the detector ID
    output_filepath = f"/home/tzahid/Desktop/Xie/data/{detector_id}/"

    # Save the coalesced DataFrame as a CSV file with headers
    coalesced_df.write.option("header", True).mode("overwrite").csv(output_filepath)

    # Stop the SparkSession
    spark.stop()

if __name__ == "__main__":
    # Read the detector IDs from the CSV file
    detector_ids = []
    with open('/sensor_list - bugatti.nvfast.orgDetectors/I-15 NB.txt', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            detector_ids.append(row[0])

    # Process each detector ID sequentially
    for detector_id in detector_ids:
        process_detector_id(detector_id)
